# Лабораторная на Классификацию ||

В ходе этой работы мы проведём классификацию на реальных данных при помощи логистической регрессии

## Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

## Получим данные и сразу их предобработаем (данные не отличаются от блокнота 1_2 - тот же набор, соответственно весь анализ был уже проведён)

In [6]:
df = pd.read_csv('./data/bank-additional-full.csv', sep=';')

# Разделим данные на категориальные и числовые
numerical_cols = ['age', 'duration', 'campaign', 'pdays', 'previous',
                  'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
                  'euribor3m', 'nr.employed']
categorical_cols = ['job', 'marital', 'education', 'default', 'housing',
                    'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# Обработаем пропуски и заменим 'unknown' на наиболее частое значение
for col in categorical_cols:
    if (df[col] == 'unknown').sum() > 0:
        mode_value = df[col][df[col] != 'unknown'].mode()[0]
        df[col] = df[col].replace('unknown', mode_value)

# Проведём One-Hot кодирование для категориальных признаков
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Стандартизируем числовые признаки
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

## Обучим базовую модель логистической регрессии

In [20]:
# Разделение на признаки и целевую переменную
X_numerical = df_encoded[numerical_cols]
y = df['y']

# Разделим на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_numerical, y, test_size=0.2, random_state=42, stratify=y
)

# Создадим baseline логистической регрессии на числовых признаках
baseline_model = LogisticRegression(
    random_state=42,
    max_iter=2000,
    class_weight='balanced'
)
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          no       0.98      0.85      0.91      7310
         yes       0.43      0.89      0.58       928

    accuracy                           0.85      8238
   macro avg       0.71      0.87      0.75      8238
weighted avg       0.92      0.85      0.87      8238



У нас вышел сильный дисбаланс классов (отказов куда больше, нежели положительных результатов), а так же низкая точность для "yes" - когда модель предсказывать "yes", то ошибается БОЛЕЕ ЧЕМ НА 50% (это критично) (модель жертвует точностью ради полноты). Но в целом после обучения на числовых признаках мы получили неплохие результаты, теперь можно улучшать модельку дальше (в частности разобраться с низким precision для "yes")

## Улучшим качество модели